In [1]:
# install libraries
# !pip install hazm
# !pip install pandas
# !pip install gensim
# !pip install tqdm
# !pip3 install sklearn

In [2]:
# libraries
import codecs
import os
import json
import pickle
import tqdm
from collections import Counter
from nltk import FreqDist
import pandas as pd
import numpy as np
import itertools
from scipy import sparse
from hazm import *
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
import json

DIRNAME = '../dataset'
data = []
for i in range(1, 8):
    with open(f'{DIRNAME}/namnak-p{i}.json', 'r', encoding="utf-8") as f:
        data.extend(json.loads(f.read()))
    with open(f'{DIRNAME}/hidoctor-p{i}.json', 'r', encoding="utf-8") as f:
        data.extend(json.loads(f.read()))


In [4]:
len(data)

4322

### normalization

In [5]:
# normalization
from hazm import *

normalizer = Normalizer()

normalized_data = []
for item in tqdm.tqdm(data):
    normalized_data.append({"title" : normalizer.normalize(item['title']), "text":normalizer.normalize(item['text']), "link": item['link']})

100%|█████████████████████████████████████████████████████████████████████████████| 4322/4322 [00:06<00:00, 662.58it/s]


In [6]:
import random
print(random.sample(normalized_data, 10)[0]['text'][:1000])

تغییرات فصل بر روی عملکرد مغز و روحیات و شخصیت افراد بسیار تاثیر گذار است. برای مثال در فصل زمستان برخی افراد دچار افسردگی می‌شوند. آیا می‌دانستید که ممکن است در یک فصل حالتان خوب باشد اما برعکس، در فصل دیگر حوصله نداشته باشید؟ در واقع عملکرد ذهنی در فصل‌های مختلف دچار تغییر می‌شود. مطالعات نشان می‌دهد که با وارد شدن به فصل جدید، ذهن به تغییرات و تحولات اطراف خود پاسخ می‌دهد که گاهی اوقات این پاسخ مثبت است و گاهی اوقات چندان مثبت نیست. احتمالا شما هم این حس‌های درونی را تجربه کرده باشید و بدانید که راجع به چه چیزی صحبت می‌کنیم. در این قسمت از نمناک قصد داریم راجع به نوسانات فصلی که روی وضعیت ذهنی تاثیرگذار است، صحبت کنیم. جهت و زاویه زمین با خورشید روی همه چیز از جمله رفتار و شخصیت ما اثر گذار است. همچنین اختلالات فصلی یا افسردگی‌های فصلی هم وجود دارند که می‌توانند مغز انسان‌ها را دچار تغییر و تحولاتی بکنند و روی روحیه آن‌ها اثرگذار باشند. این موضوع کاملا از نظر علمی به اثبات رسیده است. به همین دلیل است که درمان‌های مختلفی از جمله نور درمانی وجود دارد که در این روش، انسان‌ها جلوی آینه 

### stopwords

In [7]:
# Persian Stopwords
# https://github.com/sobhe/hazm/blob/master/hazm/data/stopwords.dat
stopwords = [normalizer.normalize(x.strip()) for x in codecs.open('stopwords.txt','r','utf-8').readlines()]

In [8]:
def create_dataframe_freq(data, stopwords=[]):
    tokens = []
    for item in tqdm.tqdm(data):
        tokens.extend([_ for _ in word_tokenize(item['text'])  if _ not in stopwords])
    return pd.DataFrame(FreqDist(tokens).most_common(30)), tokens

In [9]:
freq_analysis, tokens = create_dataframe_freq(normalized_data, stopwords)

100%|█████████████████████████████████████████████████████████████████████████████| 4322/4322 [00:13<00:00, 320.60it/s]


In [10]:
freq_analysis

,0,1
0,.,135766
1,،,98207
2,:,18986
3,استفاده,15644
4,مصرف,13151
5,بدن,12752
6,کاهش,9786
7,درمان,9637
8,پوست,8233
9,آب,8015


In [11]:
custom_stop_words = [normalizer.normalize(x.strip()) for x in codecs.open('custom_stopwords.txt','r','utf-8').readlines()]

In [12]:
total_stop_words = custom_stop_words + stopwords

In [13]:
freq_analysis, tokens = create_dataframe_freq(normalized_data, total_stop_words)

100%|█████████████████████████████████████████████████████████████████████████████| 4322/4322 [00:12<00:00, 345.82it/s]


In [14]:
freq_analysis

,0,1
0,استفاده,15644
1,مصرف,13151
2,بدن,12752
3,کاهش,9786
4,درمان,9637
5,پوست,8233
6,آب,8015
7,کمک,7871
8,بیماری,7659
9,صورت,7647


In [15]:
all_doc_tokens = []
for item in tqdm.tqdm(normalized_data):
    all_doc_tokens.append([_ for _ in word_tokenize(item['text'])  if _ not in total_stop_words])

100%|█████████████████████████████████████████████████████████████████████████████| 4322/4322 [00:13<00:00, 328.94it/s]


## Lematize

In [16]:
stemmer = Stemmer()
lemmatizer = Lemmatizer()

def get_lemma_set(tok, opt=1):
    if opt ==1:
        return stemmer.stem(tok)
    if opt ==2:
        return lemmatizer.lemmatize(tok)

In [17]:
opt = 2
tokens_after_lem = []

for tokens in all_doc_tokens:
    tokens_after_lem.append([get_lemma_set(token, opt) for token in tokens])

## tfidf

In [32]:
tfidfvectorizer = TfidfVectorizer(analyzer='word', stop_words=total_stop_words, ngram_range=(1,3), min_df=2)
doc_term_mat = tfidfvectorizer.fit_transform([' '.join(doc) for doc in tokens_after_lem])

In [33]:
doc_term_mat.shape

(4322, 349214)

In [34]:
# save tfidf vectorizer
with open('../models/tfidfvectorizer.pk', 'wb') as fin:
    pickle.dump(tfidfvectorizer, fin)

In [35]:
# load tfidf vectorizer
with open('../models/tfidfvectorizer.pk', 'rb') as fin:
    tfidfvectorizer_loaded = pickle.load(fin)

In [36]:
tfidfvectorizer_loaded.transform(['ریزش مو در مردان'])

<1x349214 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [37]:
# save matrix
os.makedirs('../models', exist_ok=True)
sparse.save_npz("../models/doc-term-tfidf.npz", doc_term_mat)

In [38]:
# save doc data
data = []
for i, item in enumerate(normalized_data):
    data.append({'title': item['title'], 'link':item['link']})
with open(f'../models/doc-term-tfidf-data.json', 'w', encoding="utf-8") as f:
    json.dump(data, f)

In [39]:
# load matrix
doc_term_mat = sparse.load_npz("../models/doc-term-tfidf.npz")

In [40]:
doc_term_mat.shape

(4322, 349214)

In [27]:
class TfidfModel:
    def __init__(self):
        self.doc_term_mat = sparse.load_npz("../models/doc-term-tfidf.npz")
        with open(f'../models/doc-term-tfidf-data.json', 'r', encoding="utf-8") as f:
            self.docs_data = json.loads(f.read())
        with open('../models/tfidfvectorizer.pk', 'rb') as fin:
            self.tfidfvectorizer = pickle.load(fin)
        self.normalizer = Normalizer()
        stopwords = [self.normalizer.normalize(x.strip()) for x in codecs.open('stopwords.txt','r','utf-8').readlines()]
        custom_stop_words = [self.normalizer.normalize(x.strip()) for x in codecs.open('custom_stopwords.txt','r','utf-8').readlines()]
        self.total_stop_words = custom_stop_words + stopwords
    
    def print_similars(self, query, k=10):
        ls = self.get_query(query, k)
        for i, item in enumerate(ls):
            print(f'{i + 1}- title: {item[0]}')
            print(f'{i + 1}- link: {item[1]}')
            print('-------------------------')
    
    def get_query(self, query, k=10):
        query_tokens = [_ for _ in word_tokenize(self.normalizer.normalize(query)) if _ not in self.total_stop_words]
        emb = self.tfidfvectorizer.transform([' '.join(query_tokens)])
        docs = []
        for i, doc in enumerate(self.doc_term_mat):
            docs.append({'title': self.docs_data[i]['title'], 'emb':doc[0].toarray()[0], 'link':self.docs_data[i]['link']})
        return self.nearest_neighbor(emb[0].toarray()[0], docs, k)
    
    def cosine_similarity(self, vector_1: np.ndarray, vector_2: np.ndarray) -> float:
        return np.dot(vector_1, vector_2)/(np.linalg.norm(vector_1) *
                                          np.linalg.norm(vector_2))
    
    def nearest_neighbor(self, v, doc_embs, k):
        data = {}
        for doc in doc_embs:
            data[doc['title']] = (self.cosine_similarity(v, doc['emb']), doc['link'])
        return [(k,v[1]) for k, v in sorted(data.items(), key=lambda item: item[1][0])][::-1][:k]

In [28]:
TfidfModel().print_similars('ویروس کرونا')

1- title: علائم ویروس کرونا در کودکان چه تفاوتی دارد؟
1- link: https://www.hidoctor.ir/352565_%d8%b9%d9%84%d8%a7%d8%a6%d9%85-%d9%88%db%8c%d8%b1%d9%88%d8%b3-%da%a9%d8%b1%d9%88%d9%86%d8%a7-%d8%af%d8%b1-%da%a9%d9%88%d8%af%da%a9%d8%a7%d9%86-%da%86%d9%87-%d8%aa%d9%81%d8%a7%d9%88%d8%aa%db%8c-%d8%af.html/
-------------------------
2- title: در مورد بیماری کرونا ویروس چه می‌دانید؟
2- link: https://www.hidoctor.ir/346899_%d8%af%d8%b1-%d9%85%d9%88%d8%b1%d8%af-%d8%a8%db%8c%d9%85%d8%a7%d8%b1%db%8c-%da%a9%d8%b1%d9%88%d9%86%d8%a7-%d9%88%db%8c%d8%b1%d9%88%d8%b3-%da%86%d9%87-%d9%85%db%8c-%d8%af%d8%a7%d9%86%db%8c%d8%af%d8%9f.html/
-------------------------
3- title: چطور بفهمم دلیل سردردم کروناست؟
3- link: https://namnak.com/headache-coronavirus-symptom.p81893
-------------------------
4- title: شایع‌ترین علائم پس از ابتلا به کرونا / چه زمانی باید درخواست کمک کنیم؟
4- link: https://www.hidoctor.ir/354939_%d8%b4%d8%a7%db%8c%d8%b9-%d8%aa%d8%b1%db%8c%d9%86-%d8%b9%d9%84%d8%a7%d8%a6%d9%85-%d9%be%d8%b3-%d8%a

In [34]:
TfidfModel().print_similars('دندان درد')

1- title: علت دندان درد در هواپیما و راه‌های تسکین درد دندان در سفرهای هوایی
1- link: https://www.hidoctor.ir/355939_%d8%b9%d9%84%d8%aa-%d8%af%d9%86%d8%af%d8%a7%d9%86-%d8%af%d8%b1%d8%af-%d8%af%d8%b1-%d9%87%d9%88%d8%a7%d9%be%db%8c%d9%85%d8%a7-%d9%88-%d8%b1%d8%a7%d9%87-%d9%87%d8%a7%db%8c-%d8%aa%d8%b3%da%a9%db%8c%d9%86.html/
-------------------------
2- title: علت اصلی انواع درد و مشکلات دندان و لثه
2- link: https://namnak.com/مشکل-دندان.p51702
-------------------------
3- title: این افراد بیشتر مراقب پوسیدگی دندان باشند!
3- link: https://namnak.com/پوسیدگی-دندان.p143
-------------------------
4- title: رازهای فوق العاده جذاب درباره دندان‌ها
4- link: https://namnak.com/facts-about-teeth.p79756
-------------------------
5- title: این درد و حالت در دندان شما یعنی عصب کشی لازم دارید
5- link: https://namnak.com/عصب-کشی-دندان.p9578
-------------------------
6- title: روش‌های سریع برای درمان گیاهی و خانگی «حساسیت دندان»
6- link: https://namnak.com/دکترعلت-حساسیت-دندان.p19355
-------------------

In [35]:
TfidfModel().print_similars('ریزش مو')

1- title: درمان ریزش مو با نسخه‌های گیاهی هندی‌ها
1- link: https://namnak.com/درمان-ریزش-مو.p23076
-------------------------
2- title: راه‌های درمان ریزش مو بعد از کرونا
2- link: https://www.hidoctor.ir/357200_%d8%b1%d8%a7%d9%87%e2%80%8c%d9%87%d8%a7%db%8c-%d8%af%d8%b1%d9%85%d8%a7%d9%86-%d8%b1%db%8c%d8%b2%d8%b4-%d9%85%d9%88-%d8%a8%d8%b9%d8%af-%d8%a7%d8%b2-%da%a9%d8%b1%d9%88%d9%86%d8%a7.html/
-------------------------
3- title: چهار تا از بهترین شامپو‌ها برای جلوگیری از ریزش مو
3- link: https://www.hidoctor.ir/354908_%da%86%d9%87%d8%a7%d8%b1-%d8%aa%d8%a7-%d8%a7%d8%b2-%d8%a8%d9%87%d8%aa%d8%b1%db%8c%d9%86-%d8%b4%d8%a7%d9%85%d9%be%d9%88-%d9%87%d8%a7-%d8%a8%d8%b1%d8%a7%db%8c-%d8%ac%d9%84%d9%88%da%af%db%8c%d8%b1%db%8c.html/
-------------------------
4- title: نکات مهم برای جلوگیری از ریزش مو در خانم‌ها و آقایان
4- link: https://www.hidoctor.ir/355441_%d9%86%da%a9%d8%a7%d8%aa-%d9%85%d9%87%d9%85-%d8%a8%d8%b1%d8%a7%db%8c-%d8%ac%d9%84%d9%88%da%af%db%8c%d8%b1%db%8c-%d8%a7%d8%b2-%d8%b1%db%8c%d8%b2%

In [36]:
TfidfModel().print_similars('مواد مخدر')

1- title: ردپای مواد مخدر روی مغز نوجوانان!!
1- link: https://namnak.com/drugs-and-teens.p45658
-------------------------
2- title: اعتیاد به این مواد مخدر ترک ندارد!
2- link: https://namnak.com/اعتیاد-به-این-مواد-مخدر-ترک-ندارد.p20238
-------------------------
3- title: مواد مخدرهای ویرانگر با تاثیرات باورنکردنی
3- link: https://namnak.com/مواد-مخدر.p61060
-------------------------
4- title: درباره مواد مخدر تسبیح چه میدانید
4- link: https://namnak.com/مواد-مخدر-تسبیح.p32839
-------------------------
5- title: عوامل موثر دراعتیاد جوانان به مواد مخدر
5- link: https://namnak.com/مسیر-پر-خطر-اعتیاد.p43562
-------------------------
6- title: کراتوم چیست؟ تعریف، تاریخچه و فواید کراتوم
6- link: https://www.hidoctor.ir/333316_%da%a9%d8%b1%d8%a7%d8%aa%d9%88%d9%85.html/
-------------------------
7- title: اکثر قرصهای خواب، لاغرکننده و اشتهاآور با مواد مخدر تهیه می‌شود
7- link: https://www.hidoctor.ir/529_%d8%a7%d9%83%d8%ab%d8%b1-%d9%82%d8%b1%d8%b5%d9%87%d8%a7%d9%8a-%d8%ae%d9%88%d8%a7%d8%a8-%d8